In [229]:
import numpy as np
from tqdm import tqdm

In [230]:
# Problem: You need to find the correct ordering of a list of numbers
# at each step, you know how many numbers are in the right place, but nothing else.

# Look up mastermind AI for ideas
# This is probably a solved problem.

# Mastermind: Five-guess algorithm for worst-case; Donald Knuth
# http://www.cs.uni.edu/~wallingf/teaching/cs3530/resources/knuth-mastermind.pdf
# Start with set S of all possible codes (length^4), length! in our case
# Start with initial guess 1122 (duplicates are not allowed in our setup)
#     Note! Because we can't repeat guesses, every initial guess is provably equally valid.
# Remove from S any option that is incompatible with the observed responses
# Select the guess that minimizes the maximum number of options in S that can remain valid after guessing it
# (this means that, assuming the worst-case scenario, we want to eliminate as much of S as possible)
#     ^^ The guess itself may not be in S, mind you. S is simply the set we want to reduce.
# Repeat. Basically just apply the Minimax algorithm.

# Let's try this algorithm. We can't repeat guesses, but other than that it should be viable.
# A bit difficult computationally.
# For each possible guess, after each guess:
# Check against each previous guess
# Does score match with number of matches? If so, match is possible.
# We can try this alone as a naive algorithm.

# Past that, here's how we can calculate worst-case score.
# For each guess in S, assume that the solution is any other guess in S
# Bin them by score.
# Size of the largest bin is the worst-case size.

# Alternatively:
# Could a genetic algorithm perform well?
# There do seem to be papers on it - as I suspected, scalability is its main attractive feature.
# 
'''

  https://www.ics.uci.edu/~goodrich/pubs/ipl.pdf
  On the Algorithmic Complexity of the Mastermind
  Game with Black-Peg Results
   - This is a similar task - the core difference is that we know that there are not two of any peg.
'''
#
pass

In [231]:
# Run a simulation
def runSimulation(size, sol): # Take a size and a solution class
  solution = np.arange(size)
  np.random.shuffle(solution)
  model = sol(size)
  attempts = 0
  attempt = None
  prev_score = None
  while (True):
    attempts += 1
    attempt = model.predict(prev_score)
    prev_score = np.sum(attempt==solution)
    if (prev_score == size):
      break
  return attempts # number of attempts to reach a correct solution
    
def avgSimulation(size, sol, trials=1000):
  sum = 0
  for i in tqdm(range(trials)):
    sum += runSimulation(size, sol)
  return sum / trials

In [214]:
# Generic class for a solution.
# Has one function, to make a prediction.
class Solution:
  def __init__(self, size): # Init only takes the size of the set we're working with
    self.size = size
    pass
  def predict(self, prev_score): # Takes the score of the previous prediction.
    pred = np.range(self.size)
    np.random.shuffle(pred)
    return pred # make a random guess

# Iterate through each possible ordering
# Averages 360 tries for a trial of size 6
class ExhaustiveSolution(Solution):
  def __init__(self, size): # Init only takes the size of the set we're working with
    self.size = size
    self.count = 0
    self.num_pos = np.math.factorial(self.size)
    self.range = list(range(size))
  def predict(self, prev_score): # Takes the score of the previous prediction.
    prediction = []
    x = self.size
    rem = self.count
    l = self.range.copy()
    while (x > 0):
      f = np.math.factorial(x-1)
      ix = rem // f
      prediction.append(l[ix])
      del l[ix]
      rem = rem % f
      x -= 1
    self.count += 1
    return np.array(prediction)

def populate_array(start, rem): # Take the first part of an array and all remaining numbers to be added
    # For each value in rem, add it to start, 
    result = []
    if (len(rem)==0):
      return [start]
    for i in range(len(rem)):
      r = rem.copy()
      st = start.copy()
      st.append(r[i])
      del r[i]
      result.extend(populate_array(st, r))
    return result

# Eliminate impossible guesses, pick arbitrarily from those that remain.
# Average 26.388 tries for a trial of size 6
class NaiveKnuth(Solution):
  def __init__(self, size): # Init only takes the size of the set we're working with
    self.size = size
    self.count = 0
    self.range = list(range(size))
    self.solutions = np.array(populate_array([], list(range(size)))) # Every possible permutation
    self.prev_solution = None

  def predict(self, prev_score): # Takes the score of the previous prediction.
    if (prev_score is not None):
      # If the number of common values is less than the score, discard.
      common_values = np.sum(self.prev_solution==self.solutions, axis=1)
      # Must have exactly as many differences as points
      valid_solutions = common_values == prev_score 
      self.solutions = self.solutions[valid_solutions]
    self.prev_solution = self.solutions[0]
    return self.prev_solution

def getMaxBin(index, solutions, all_solutions):
  s = all_solutions[index]
  k = solutions.copy()
  scores = np.sum(solutions==s, axis=1)
  _, b = np.unique(scores, return_counts=True)
  return np.max(b) # The max count is the score. We want the minimum score
  #return np.sum(b**2) / np.sum(b) # Target expected count?

def getBestIndex(solutions, all_solutions, possible):
  minCost = np.inf
  minIx = -1
  for i in range(len(all_solutions)):
    cost = getMaxBin(i, solutions, all_solutions)
    if (cost < minCost or ((cost == minCost) and (possible[i]==True))):
      minCost = cost
      minIx = i
  return minIx

# Same as above, except we take every remaining solution and every possible score
# and bin the remaining solutions based on score
# and then minimize the size of the largest bin
# We can also guess 
# Average x tries for a trial of size 6
class Knuth(Solution):
  def __init__(self, size): # Init only takes the size of the set we're working with
    self.size = size
    self.count = 0
    self.range = list(range(size))
    self.solutions = np.array(populate_array([], list(range(size)))) # Every possible permutation
    self.all_solutions = self.solutions.copy()
    self.prev_solution = None
    self.prev_ix = None
    self.possible = np.full(len(self.solutions), True)
    self.pos_ix = np.arange(len(self.solutions))

  def predict(self, prev_score): # Takes the score of the previous prediction.
    if (prev_score is not None): # Clear eliminated solutions
      # If the number of common values is less than the score, discard.
      common_values = np.sum(self.prev_solution==self.solutions, axis=1)
      # Must have exactly as many differences as points
      valid_solutions = common_values == prev_score 
      #
      self.possible[self.pos_ix[np.invert(valid_solutions)]] = False
      self.pos_ix = self.pos_ix[valid_solutions] 
      #
      self.solutions = self.solutions[valid_solutions]
    # Now that our solution set is filtered, get the best index.
    self.prev_ix = getBestIndex(self.solutions, self.all_solutions, self.possible)
    self.count += 1
    #
    self.prev_solution = self.all_solutions[self.prev_ix]
    return self.prev_solution

In [237]:
#avgSimulation(6, ExhaustiveSolution) # 360
avgSimulation(6, Knuth, trials=10000)

100%|██████████| 10000/10000 [37:34<00:00,  4.44it/s]


5.6592

In [236]:
avgSimulation(6, NaiveKnuth, trials=10000)

100%|██████████| 10000/10000 [00:28<00:00, 346.42it/s]


5.8899